In [1]:
from llm.llm.chatgpt import ChatGPT
from setup_db import create_chroma_db, connect_to_db, execute_query, DBHUB
from llm_test import reasoning_text2SQL, CoT_reasoning
from llm_general import  find_suitable_row_v2

In [2]:
llm = ChatGPT()

In [3]:
question = "What is the average total assets of bank in Q2 2024"

In [4]:
find_suitable_row_v2(llm, question, verbose=True)

CompletionUsage(completion_tokens=31, prompt_tokens=165, total_tokens=196, prompt_tokens_details={'cached_tokens': 0}, completion_tokens_details={'reasoning_tokens': 0})
Find suitable column response: 
```json
{
    "industry": ["Banking"],
    "financial_statement_row": ["Total Assets"],
    "financial_ratio_row": []
}
```


{'industry': ['Banking'],
 'financial_statement_row': ['Total Assets'],
 'financial_ratio_row': []}

In [15]:
db_name = 'test_db'
user = 'postgres'
password = '12345678'
port = '5433'
host = 'localhost'

In [16]:
conn = connect_to_db(db_name, user, password, host, port)


Connecting to database test_db, postgres...


In [17]:
collection_chromadb = 'category_bank_chroma'
persist_directory = 'data/category_bank_chroma'
bank_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'category_non_bank_chroma'
persist_directory = 'data/category_non_bank_chroma'
none_bank_vector_store = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'company_name_chroma'
persist_directory = 'data/company_name_chroma'
vector_db_company = create_chroma_db(collection_chromadb, persist_directory)

collection_chromadb = 'sql_query'
persist_directory = 'data/sql_query'
vector_db_sql = create_chroma_db(collection_chromadb, persist_directory)

In [18]:
db = DBHUB(conn, bank_vector_store, none_bank_vector_store, vector_db_company, vector_db_sql)

In [19]:
prompt = "Calculate ROA, ROE of the bank VIB in financial year 2023"

In [20]:
sql = reasoning_text2SQL(llm, prompt, db, 5, verbose=True)

CompletionUsage(completion_tokens=40, prompt_tokens=166, total_tokens=206, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0), prompt_tokens_details={'cached_tokens': 0})
Find suitable column response: 
```json
{
    "bank_column_name": [
        "Net Income",
        "Total Assets",
        "Shareholders' Equity"
    ],
    "non_bank_column_name": []
}
```
Bank column:    category_code                                         en_caption
0         IS_003                                Net interest income
1         IS_006          Net profit/(loss) from service activities
2         IS_012            Net profit/(loss) from other activities
3         IS_021                                   Profit after tax
4         IS_023                           Basic earnings per share
5         BS_220                                       Fixed assets
6         BS_250                                       Other assets
7         BS_254                                       Other ass

In [21]:
sql

[{'language': 'sql',
  'code': "-- Calculate ROA and ROE for bank VIB in financial year 2023\n\nWITH net_profit AS (\n    SELECT data AS net_profit\n    FROM bank_financial_report\n    WHERE stock_code = 'VIB'\n      AND year = 2023\n      AND quarter = 0\n      AND category_code = 'IS_021'\n),\n\ntotal_assets AS (\n    SELECT data AS total_assets\n    FROM bank_financial_report\n    WHERE stock_code = 'VIB'\n      AND year = 2023\n      AND quarter = 0\n      AND category_code = 'BS_300'\n),\n\nshareholders_equity AS (\n    SELECT data AS shareholders_equity\n    FROM bank_financial_report\n    WHERE stock_code = 'VIB'\n      AND year = 2023\n      AND quarter = 0\n      AND category_code = 'BS_500'\n)\n\nSELECT \n    np.net_profit,\n    ta.total_assets,\n    se.shareholders_equity,\n    (np.net_profit / ta.total_assets) AS ROA,\n    (np.net_profit / se.shareholders_equity) AS ROE\nFROM net_profit np\nJOIN total_assets ta ON 1=1\nJOIN shareholders_equity se ON 1=1;\n"}]

In [22]:
print(sql[-1]['code'])

-- Calculate ROA and ROE for bank VIB in financial year 2023

WITH net_profit AS (
    SELECT data AS net_profit
    FROM bank_financial_report
    WHERE stock_code = 'VIB'
      AND year = 2023
      AND quarter = 0
      AND category_code = 'IS_021'
),

total_assets AS (
    SELECT data AS total_assets
    FROM bank_financial_report
    WHERE stock_code = 'VIB'
      AND year = 2023
      AND quarter = 0
      AND category_code = 'BS_300'
),

shareholders_equity AS (
    SELECT data AS shareholders_equity
    FROM bank_financial_report
    WHERE stock_code = 'VIB'
      AND year = 2023
      AND quarter = 0
      AND category_code = 'BS_500'
)

SELECT 
    np.net_profit,
    ta.total_assets,
    se.shareholders_equity,
    (np.net_profit / ta.total_assets) AS ROA,
    (np.net_profit / se.shareholders_equity) AS ROE
FROM net_profit np
JOIN total_assets ta ON 1=1
JOIN shareholders_equity se ON 1=1;



In [23]:
db.query(sql[-1]['code'], return_type='dataframe')

,net_profit,total_assets,shareholders_equity,roa,roe
0,8563053,409881373,37939587,0.02089153975777279345,0.22570232512019701216


In [24]:
res = bank_vector_store.similarity_search('Asset', 5)
res

[Document(page_content='Other assets', metadata={'code': 'BS_250', 'lang': 'en'}),
 Document(page_content='Other assets', metadata={'code': 'BS_254', 'lang': 'en'}),
 Document(page_content='Fixed assets', metadata={'code': 'BS_220', 'lang': 'en'}),
 Document(page_content='Asset revaluation difference', metadata={'code': 'BS_440', 'lang': 'en'}),
 Document(page_content='Other capital', metadata={'code': 'BS_416', 'lang': 'en'})]

: 